In [4]:
import torch
import torch.nn as nn
import numpy as np
import time
import math
from matplotlib import pyplot
import pandas as pd

torch.manual_seed(0)
np.random.seed(0)

# S is the source sequence length
# T is the target sequence length
# N is the batch size
# E is the feature number

#src = torch.rand((10, 32, 512)) # (S,N,E)
#tgt = torch.rand((20, 32, 512)) # (T,N,E)
#out = transformer_model(src, tgt)

input_window = 150  # number of input steps
output_window = 1  # number of prediction steps, in this model its fixed to one
batch_size = 512
device = torch.device("cuda")


class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=6000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2).float() *
            (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        #pe.requires_grad = False
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:x.size(0), :]


class TransAm(nn.Module):
    def __init__(self, feature_size=300, num_layers=1, dropout=0.1):
        super(TransAm, self).__init__()
        self.model_type = 'Transformer'

        self.src_mask = None
        self.pos_encoder = PositionalEncoding(feature_size)
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=feature_size,
                                                        nhead=10,
                                                        dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer,
                                                         num_layers=num_layers)
        self.decoder = nn.Linear(feature_size, 1)
        self.init_weights()

    def init_weights(self):
        initrange = 0.1
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src):
        if self.src_mask is None or self.src_mask.size(0) != len(src):
            device = src.device
            mask = self._generate_square_subsequent_mask(len(src)).to(device)
            self.src_mask = mask

        src = self.pos_encoder(src)
        output = self.transformer_encoder(src,
                                          self.src_mask)  #, self.src_mask)
        output = self.decoder(output)
        return output

    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(
            mask == 1, float(0.0))
        return mask


# if window is 100 and prediction step is 1
# in -> [0..99]
# target -> [1..100]
def create_inout_sequences(input_data, tw):
    inout_seq = []
    L = len(input_data)
    for i in range(L - tw):
        train_seq = input_data[i:i + tw]
        train_label = input_data[i + output_window:i + tw + output_window]
        inout_seq.append((train_seq, train_label))
    return torch.FloatTensor(inout_seq)


def get_data(name):
    # construct a littel toy dataset
    time = np.arange(0, 400, 0.1)
    data = pd.read_csv('../data/399300.csv')
    amplitude = data[name].values
    from sklearn.preprocessing import MinMaxScaler

    #loading weather data from a file
    #from pandas import read_csv
    #series = read_csv('daily-min-temperatures.csv', header=0, index_col=0, parse_dates=True, squeeze=True)

    # looks like normalizing input values curtial for the model
    scaler = MinMaxScaler(feature_range=(-1, 1))
    #amplitude = scaler.fit_transform(series.to_numpy().reshape(-1, 1)).reshape(-1)
    amplitude = scaler.fit_transform(amplitude.reshape(-1, 1)).reshape(-1)

    sampels = int(len(amplitude)*0.9)
    train_data = amplitude[:sampels]
    test_data = amplitude[sampels:]

    # convert our train data into a pytorch train tensor
    #train_tensor = torch.FloatTensor(train_data).view(-1)
    # todo: add comment..
    train_sequence = create_inout_sequences(train_data, input_window)
    train_sequence = train_sequence[:
                                    -output_window]  #todo: fix hack? -> din't think this through, looks like the last n sequences are to short, so I just remove them. Hackety Hack..

    #test_data = torch.FloatTensor(test_data).view(-1)
    test_data = create_inout_sequences(test_data, input_window)
    test_data = test_data[:-output_window]  #todo: fix hack?

    return train_sequence.to(device), test_data.to(device),scaler


def get_batch(source, i, batch_size):
    seq_len = min(batch_size, len(source) - 1 - i)
    data = source[i:i + seq_len]
    input = torch.stack(
        torch.stack([item[0] for item in data]).chunk(input_window,1))  # 1 is feature size
    target = torch.stack(
        torch.stack([item[1] for item in data]).chunk(input_window, 1))
    return input, target


def train(train_data,epoch):
    model.train()  # Turn on the train mode \o/
    total_loss = 0.
    start_time = time.time()
    

    for batch, i in enumerate(range(0, len(train_data) - 1, batch_size)):
        data, targets = get_batch(train_data, i, batch_size)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, targets)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.7)
        optimizer.step()

        total_loss += loss.item()
        log_interval = int(len(train_data) / batch_size)
        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | '
                  'lr {:02.6f} | {:5.2f} ms | '
                  'loss {:5.5f} | ppl {:8.2f}'.format(
                      epoch, batch,
                      len(train_data) // batch_size,
                      scheduler.get_lr()[0], elapsed * 1000 / log_interval,
                      cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()


def plot_and_loss(eval_model, data_source, epoch):
    eval_model.eval()
    total_loss = 0.
    test_result = torch.Tensor(0)
    truth = torch.Tensor(0)
    with torch.no_grad():
        for i in range(0, len(data_source) - 1):
            data, target = get_batch(data_source, i, 1)
            output = eval_model(data)
            total_loss += criterion(output, target).item()
            test_result = torch.cat((test_result, output[-1].view(-1).cpu()),
                                    0)
            truth = torch.cat((truth, target[-1].view(-1).cpu()), 0)

    #test_result = test_result.cpu().numpy() -> no need to detach stuff..
    len(test_result)

    pyplot.plot(test_result, color="red", label="prediction")
    pyplot.plot(truth[:len(test_result)], color="blue", label="truth")
    pyplot.plot(test_result - truth, color="green", label="error")
    pyplot.legend()
    pyplot.grid(True, which='both')
    pyplot.axhline(y=0, color='k')
    pyplot.savefig('graph/transformer-epoch%d.png' % epoch)
    pyplot.close()

    return total_loss / i


# predict the next n steps based on the input data
def predict_future(eval_model, data_source, steps):
    eval_model.eval()
    total_loss = 0.
    test_result = torch.Tensor(0)
    truth = torch.Tensor(0)
    data, _ = get_batch(data_source, 0, 1)
    with torch.no_grad():
        for i in range(0, steps):
            output = eval_model(data[-input_window:])
            data = torch.cat((data, output[-1:]))

    data = data.cpu().view(-1)

    # I used this plot to visualize if the model pics up any long therm structure within the data.
    pyplot.plot(data, color="red")
    pyplot.plot(data[:input_window], color="blue")
    pyplot.grid(True, which='both')
    pyplot.axhline(y=0, color='k')
    pyplot.savefig('graph/transformer-future%d.png' % steps)
    pyplot.close()


def evaluate(eval_model, data_source):
    eval_model.eval()  # Turn on the evaluation mode
    total_loss = 0.
    eval_batch_size = 1000
    with torch.no_grad():
        for i in range(0, len(data_source) - 1, eval_batch_size):
            data, targets = get_batch(data_source, i, eval_batch_size)
            output = eval_model(data)
            total_loss += len(data[0]) * criterion(output,
                                                   targets).cpu().item()
    return total_loss / len(data_source)


In [5]:
def train_and_get(name):
    input_window = 120  # number of input steps
    output_window = 1  # number of prediction steps, in this model its fixed to one
    batch_size = 512
    device = torch.device("cuda")
    train_data, val_data, scaler = get_data(name)
    global model 
    model = TransAm().to(device)

    global criterion
    criterion = nn.MSELoss()
    lr = 0.001  # learning rate
    #optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    global optimizer
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    global scheduler
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.98)

    best_val_loss = float("inf")
    global epochs
    epochs = 300  # The number of epochs
    best_model = None

    for epoch in range(1, epochs + 1):
        epoch_start_time = time.time()
        train(train_data,epoch)

        if (epoch % 100 == 0):
            val_loss = plot_and_loss(model, val_data, epoch)
            predict_future(model, val_data, 5)
        else:
            val_loss = evaluate(model, val_data)

        if(epoch % 50 ==0):
            print('-' * 89)
            print(
                '| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.5f} | valid ppl {:8.2f}'
                .format(epoch, (time.time() - epoch_start_time), val_loss,
                        math.exp(val_loss)))
            print('-' * 89)
        scheduler.step()
    
    def get_predict(eval_model, data_source):
        eval_model.eval()
        total_loss = 0.
        test_result = torch.Tensor(0)
        truth = torch.Tensor(0)
        with torch.no_grad():
            for i in range(0, len(data_source) - 1):
                data, target = get_batch(data_source, i, 1)
                output = eval_model(data)
                total_loss += criterion(output, target).item()
                test_result = torch.cat((test_result, output[-1].view(-1).cpu()),
                                        0)
                truth = torch.cat((truth, target[-1].view(-1).cpu()), 0)

        #test_result = test_result.cpu().numpy() -> no need to detach stuff..
        return scaler.inverse_transform(test_result.view(-1, 1))[120:], scaler.inverse_transform(truth.view(-1, 1))[120:]

    
    return get_predict(model, val_data)


In [6]:
result = dict()
for i in ['open','close','high','low']:
    predict,truth = train_and_get(i)
    result.update({i:predict})
    result.update({i+'_truth':truth})

d:\tools\anaconda3\envs\data\lib\site-packages\torch\optim\lr_scheduler.py:381: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


| epoch   1 |     8/    8 batches | lr 0.001000 | 129.34 ms | loss 28.85596 | ppl 3403957749763.20
| epoch   2 |     8/    8 batches | lr 0.000960 | 89.48 ms | loss 4.95687 | ppl   142.15
| epoch   3 |     8/    8 batches | lr 0.000941 | 89.17 ms | loss 0.56694 | ppl     1.76
| epoch   4 |     8/    8 batches | lr 0.000922 | 88.22 ms | loss 0.65478 | ppl     1.92
| epoch   5 |     8/    8 batches | lr 0.000904 | 89.00 ms | loss 1.60253 | ppl     4.97
| epoch   6 |     8/    8 batches | lr 0.000886 | 90.30 ms | loss 0.51910 | ppl     1.68
| epoch   7 |     8/    8 batches | lr 0.000868 | 90.20 ms | loss 0.32358 | ppl     1.38
| epoch   8 |     8/    8 batches | lr 0.000851 | 90.98 ms | loss 0.12584 | ppl     1.13
| epoch   9 |     8/    8 batches | lr 0.000834 | 89.47 ms | loss 0.20405 | ppl     1.23
| epoch  10 |     8/    8 batches | lr 0.000817 | 89.88 ms | loss 0.15181 | ppl     1.16
| epoch  11 |     8/    8 batches | lr 0.000801 | 90.04 ms | loss 0.11742 | ppl     1.12
| epoch  12

d:\tools\anaconda3\envs\data\lib\site-packages\torch\optim\lr_scheduler.py:381: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


| epoch   1 |     8/    8 batches | lr 0.001000 | 88.77 ms | loss 9.68845 | ppl 16130.23
| epoch   2 |     8/    8 batches | lr 0.000960 | 89.64 ms | loss 12.62445 | ppl 303898.04
| epoch   3 |     8/    8 batches | lr 0.000941 | 89.37 ms | loss 1.12618 | ppl     3.08
| epoch   4 |     8/    8 batches | lr 0.000922 | 88.95 ms | loss 0.77268 | ppl     2.17
| epoch   5 |     8/    8 batches | lr 0.000904 | 89.24 ms | loss 0.09198 | ppl     1.10
| epoch   6 |     8/    8 batches | lr 0.000886 | 89.30 ms | loss 0.10605 | ppl     1.11
| epoch   7 |     8/    8 batches | lr 0.000868 | 90.67 ms | loss 0.22271 | ppl     1.25
| epoch   8 |     8/    8 batches | lr 0.000851 | 90.90 ms | loss 0.27169 | ppl     1.31
| epoch   9 |     8/    8 batches | lr 0.000834 | 89.86 ms | loss 0.04487 | ppl     1.05
| epoch  10 |     8/    8 batches | lr 0.000817 | 89.41 ms | loss 0.10763 | ppl     1.11
| epoch  11 |     8/    8 batches | lr 0.000801 | 89.44 ms | loss 0.03524 | ppl     1.04
| epoch  12 |     8

d:\tools\anaconda3\envs\data\lib\site-packages\torch\optim\lr_scheduler.py:381: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


| epoch   1 |     8/    8 batches | lr 0.001000 | 89.47 ms | loss 28.31386 | ppl 1979485998565.38
| epoch   2 |     8/    8 batches | lr 0.000960 | 100.44 ms | loss 5.50485 | ppl   245.88
| epoch   3 |     8/    8 batches | lr 0.000941 | 91.82 ms | loss 1.38073 | ppl     3.98
| epoch   4 |     8/    8 batches | lr 0.000922 | 90.90 ms | loss 0.48929 | ppl     1.63
| epoch   5 |     8/    8 batches | lr 0.000904 | 90.46 ms | loss 0.15500 | ppl     1.17
| epoch   6 |     8/    8 batches | lr 0.000886 | 90.14 ms | loss 0.08348 | ppl     1.09
| epoch   7 |     8/    8 batches | lr 0.000868 | 90.59 ms | loss 0.03604 | ppl     1.04
| epoch   8 |     8/    8 batches | lr 0.000851 | 91.63 ms | loss 0.02714 | ppl     1.03
| epoch   9 |     8/    8 batches | lr 0.000834 | 91.37 ms | loss 0.03488 | ppl     1.04
| epoch  10 |     8/    8 batches | lr 0.000817 | 90.87 ms | loss 0.03718 | ppl     1.04
| epoch  11 |     8/    8 batches | lr 0.000801 | 90.13 ms | loss 0.25166 | ppl     1.29
| epoch  12

d:\tools\anaconda3\envs\data\lib\site-packages\torch\optim\lr_scheduler.py:381: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


| epoch   1 |     8/    8 batches | lr 0.001000 | 89.06 ms | loss 23.02655 | ppl 10006974454.62
| epoch   2 |     8/    8 batches | lr 0.000960 | 89.43 ms | loss 0.95368 | ppl     2.60
| epoch   3 |     8/    8 batches | lr 0.000941 | 89.28 ms | loss 0.37163 | ppl     1.45
| epoch   4 |     8/    8 batches | lr 0.000922 | 89.07 ms | loss 0.09444 | ppl     1.10
| epoch   5 |     8/    8 batches | lr 0.000904 | 89.08 ms | loss 0.26842 | ppl     1.31
| epoch   6 |     8/    8 batches | lr 0.000886 | 89.09 ms | loss 0.13204 | ppl     1.14
| epoch   7 |     8/    8 batches | lr 0.000868 | 89.18 ms | loss 0.15611 | ppl     1.17
| epoch   8 |     8/    8 batches | lr 0.000851 | 89.08 ms | loss 0.12912 | ppl     1.14
| epoch   9 |     8/    8 batches | lr 0.000834 | 89.04 ms | loss 0.06987 | ppl     1.07
| epoch  10 |     8/    8 batches | lr 0.000817 | 88.93 ms | loss 0.13342 | ppl     1.14
| epoch  11 |     8/    8 batches | lr 0.000801 | 88.87 ms | loss 0.11817 | ppl     1.13
| epoch  12 | 

In [ ]:
for i in result.keys():
    result[i] = result[i].reshape(-1)
output = pd.DataFrame(result)

In [ ]:
output.to_csv('result2.csv',index=False)